In [ ]:
# !pip install catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/nto/train.csv")
test = pd.read_csv("/content/drive/MyDrive/nto/test.csv")
topics = pd.read_csv("/content/drive/MyDrive/nto/topics.csv")
users = pd.read_csv("/content/drive/MyDrive/nto/users.csv")

In [ ]:
train_size = len(train)
users_size = len(users)
topics_size = len(topics)

In [ ]:
users_id, topics_id = [0 for i in range(users_size)], [0 for i in range(topics_size)]
for i in range(len(users)):
  cur_id = users.userId[i]
  users_id[cur_id] = i
for i in range(len(topics)):
  cur_id = topics.topicId[i]
  topics_id[cur_id] = i

In [ ]:
pca5 = PCA(n_components = 5)
pca4 = PCA(n_components = 4)

txt_topics = topics
img_topics = topics

for i in topics.columns[1:101]:
  txt_topics = txt_topics.drop(i, axis=1)
for i in topics.columns[101:]:
  img_topics = img_topics.drop(i, axis=1)

In [ ]:
txt_topics, img_topics = img_topics, txt_topics

In [ ]:
txt_topics = txt_topics.drop('topicId', axis=1)
img_topics = img_topics.drop('topicId', axis=1)

In [ ]:
topics_txt_reduced = pd.DataFrame(pca4.fit_transform(txt_topics))
topics_img_reduced = pd.DataFrame(pca5.fit_transform(img_topics))

In [ ]:
users_reduced = users.drop('userId', axis=1)

In [ ]:
topics_txt_reduced.columns = ['A', 'B', 'C', 'D']

In [ ]:
mas = list(users_reduced.columns) + list(topics_txt_reduced.columns) + list(topics_img_reduced.columns)

In [ ]:
like_cnt = [0 for i in range(len(users) + 1)]
watched_cnt = [0 for i in range(len(users) + 1)]
like_rate = []
like_cnt2 = [0 for i in range(len(topics) + 1)]
watched_cnt2 = [0 for i in range(len(topics) + 1)]

train_np = train.to_numpy()
likes = []
Watches = []
for i in range(len(train)):
  watched_cnt[train_np[i][1]]+= 1
  watched_cnt2[train_np[i][0]]+= 1
  if(train_np[i][2] == 'L'):
    like_cnt[train_np[i][1]] += 1
    like_cnt2[train_np[i][0]]+=1

for i in range(len(train)):
  likes.append(like_cnt[train_np[i][1]])
  Watches.append(watched_cnt[train_np[i][1]])
train['likeCnt'] = likes
train['watchCnt'] = Watches

likes = []
Watches = []

for i in range(len(train)):
  likes.append(like_cnt[train_np[i][0]])
  Watches.append(watched_cnt[train_np[i][0]])

train['totalLiked'] = likes
train['totalWatched'] = Watches

In [ ]:
gnd = []
users_np = (users.to_numpy()).astype(int)
users_np.astype(int)
for i in range(len(users) + 1):
  gnd.append(0)
for i in range(len(users)):
  gnd[users_np[i][0]] = users_np[i][2] - 1
genders = []
for i in range(len(train)):
  genders.append(gnd[train_np[i][1]])
train['gender'] = genders

In [ ]:
likes = []
test_np = test.to_numpy()
for i in range(len(test)):
  likes.append(like_cnt[test_np[i][1]])
test['likeCnt'] = likes
watches = []
for i in range(len(test)):
  watches.append(watched_cnt[test_np[i][1]])
test['watchCnt'] = watches
likes = []
watches = []
for i in range(len(test)):
  likes.append(like_cnt2[test_np[i][0]])
  watches.append(watched_cnt2[test_np[i][0]])
test['totalLiked'] = likes
test['totalWatched'] = watches
genders = []
for i in range(len(test)):
  genders.append(gnd[test_np[i][1]])
test['gender'] = genders

In [ ]:
likes, watches, gnd, genders, like_cnt, watched_cnt, like_cnt2, watched_cnt2, train_np, users_np = [], [], [], [], [], [], [], [], [], []

In [ ]:
N = len(test)
data_train = pd.DataFrame()
data_test = pd.DataFrame()

In [ ]:
# train = train.sample(frac = 1)
# data_train.index = range(len(train))

# Users fill

In [ ]:
for j in users_reduced.columns:

  add_train, add_test = [], []

  for i in range(N):
    add_test.append(users_reduced[j][users_id[test.userId[i]]])
  for i in range(N):
    add_train.append(users_reduced[j][users_id[train.userId[i]]])

  data_train[j] = add_train
  data_test[j] = add_test

  print(j)

# for i in range(N):

#   cur_user_id_train = users_id[train['userId'][i]]
#   cur_topic_id_train = topics_id[train['topicId'][i]]
#   cur_user_id_test = users_id[test['userId'][i]]
#   cur_topic_id_test = topics_id[test['topicId'][i]]

  


# Topics fill

In [ ]:
for j in topics_txt_reduced.columns:

  add_train, add_test = [], []

  for i in range(N):
    add_test.append(topics_txt_reduced[j][topics_id[test.topicId[i]]])
  for i in range(N):
    add_train.append(topics_txt_reduced[j][topics_id[train.topicId[i]]])

  data_train[j] = add_train
  data_test[j] = add_test

  print(j)

for j in topics_img_reduced.columns:

  add_train, add_test = [], []

  for i in range(N):
    add_test.append(topics_img_reduced[j][topics_id[test.topicId[i]]])
  for i in range(N):
    add_train.append(topics_img_reduced[j][topics_id[train.topicId[i]]])

  data_train[j] = add_train
  data_test[j] = add_test

  print(j)

In [ ]:
y = train['feedback'].head(N)

# Downloading емае

In [ ]:
# data_train.to_csv("/content/drive/MyDrive/nto/saved_dataframes/data_train_5.csv", index=False)
# data_test.to_csv("/content/drive/MyDrive/nto/saved_dataframes/data_test_5.csv", index=False)

In [ ]:
# data_train = pd.read_csv("/content/drive/MyDrive/nto/saved_dataframes/data_train_5.csv")

In [ ]:
# data_test = pd.read_csv("/content/drive/MyDrive/nto/saved_dataframes/data_test_5.csv")

In [ ]:
data_train = pd.concat([data_train, train[['gender', 'feedback']]], axis=1)
data_test = pd.concat([data_test, test['gender']], axis=1)

In [ ]:
data_train = data_train.sample(frac = 1)
# data_train.index = range(len(train))

In [ ]:
y = data_train.feedback

In [ ]:
data_train = data_train.drop('feedback', axis=1)

In [ ]:
data_train['birthDate'] = data_train['birthDate'].fillna(data_train['birthDate'].mean())

In [ ]:
data_train['birthDate'] = data_train['birthDate'].astype(float)

# Catboost

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_train.head(N), y.head(N), test_size=0.2, random_state=228)

In [ ]:
model = CatBoostClassifier( iterations=500,
                            learning_rate=0.2, 
                            depth=15, 
                            eval_metric = 'F1',
                            loss_function='Logloss',
                            random_state=7)
model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

In [ ]:
ans = model.predict(data_test)

# Ans save

In [ ]:
f = open("ans.txt", "w")

In [ ]:
f_ans = list(ans)
for i in f_ans:
  f.write(i)
  f.write('\n')


In [ ]:
f.close()